In [2]:
# 读取数据
# show info of dataset
import pandas as pd

# 数据集文件路径
file_path = '/home/zhengyihao/dataset/covertype/covtype_with_key.data'

# 使用列名列表读取数据
data = pd.read_csv(file_path)
# 获取Cover_Type的值域（唯一值）
unique_values = data['Cover_Type'].unique()

# 获取每个值的计数
value_counts = data['Cover_Type'].value_counts()

# 打印结果
print("Cover_Type的值域: ", unique_values)
print("\n每个值的大小:")
print(value_counts)


Cover_Type的值域:  [5 2 1 7 3 6 4]

每个值的大小:
Cover_Type
2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: count, dtype: int64


## 嵌入水印

In [3]:
# 定义用户私钥 Ks
import secrets

# 生成 16 bits 的随机密钥
Ks = secrets.token_hex(2) # 用户私钥
k = secrets.token_hex(2) # attribute 对应的私钥

print("Generated 16 bits Ks: ", Ks)
print("Generated 16 bits k: ", k)

# 生成嵌入水印信息

import random

# 生成一个长度为 400 的字符串，它由随机的 0 和 1 组成
watermark = ''.join(random.choice('01') for _ in range(400))
# 打印二进制字符串
print("要嵌入的水印信息是",watermark)

# 统计二进制字符串中 1 的个数
count_1 = watermark.count('1')

# 统计二进制字符串中 0 的个数
count_0 = watermark.count('0')

# 打印结果
print("Count of 1: ", count_1)
print("Count of 0: ", count_0)

Generated 16 bits Ks:  b652
Generated 16 bits k:  b9e4
要嵌入的水印信息是 0010111000010110110011111100101010111011111100001001101100011110000011110110111001110110010001101101011111110110010100001110011000110010100110000110001001000010000001011010101010001000010010010011000000001011100010001001111011010001001111001011001000110111000100010100100101011110110001000100110001001011011010000111110001101010010111001110000000011101010100110000100011001100011001010111000111010010
Count of 1:  187
Count of 0:  213


In [5]:
import hashlib
import math


# watermark_interval
L = len(watermark)
gamma = data.shape[0] // L * 8
record_table = {}

for index, row in data.iterrows():
    tP_bytes = bin(row['primary_key'])[2:]
    
    # 计算 H(t.P || k)
    inner_hash = hashlib.sha256((tP_bytes + str(k)).encode()).hexdigest()
    # 计算 H(K || H(t.P || k))
    outer_hash = hashlib.sha256((str(Ks) + inner_hash).encode()).hexdigest()

    C = int(outer_hash,16) % gamma # watermark_tuple
    if C % gamma == 0:
        zeta = outer_hash % L # watermark_index

        x = row['Cover_Type']
        new_x = math.floor((x + 5)/ 2)

        if watermark[zeta] == '1':
            data.at[index,'Cover_Type'] = new_x

            # record
            key = hashlib.sha256((tP_bytes + str(k)).encode()).hexdigest()
            value = hashlib.sha256((bin(new_x)[2:] + str(k) + bin(abs(x - new_x)[2:])).encode()).hexdigest()

            record_table[key] = value
        else:
            # record
            key = hashlib.sha256(tP_bytes + str(k)).hexdigest()
            value = hashlib.sha256(bin(x)[2:] + str(k)).hexdigest()

            record_table[key] = value
    
file_name = '/home/zhengyihao/dataset/covertype/semanticmark_covertype.data.csv'
data.to_csv(file_name, index=False)
    

IndexError: invalid index to scalar variable.

In [54]:
import hashlib
import math

L = len(watermark)
gamma = 400
print(gamma)
record_table = {}

cnt = 0

for index, row in data.iterrows():
    # tP_bytes = bin(row['primary_key'])[2:]
    
    k_hash = hashlib.blake2b(f"{row['primary_key']}{k}".encode()).hexdigest()
    Ks_hash = hashlib.blake2b(f"{Ks}{k_hash}".encode()).hexdigest()
    C = int(Ks_hash,16)
    if C % gamma == 0:
        zeta = int(hashlib.blake2b(f"{Ks}{k_hash}{1}".encode()).hexdigest(),16) % L
        # print(zeta)
        x = row['Cover_Type']
        new_x = math.floor((x + 5)/ 2)
        # print(C)
        # print(x,",",new_x," ")
        if watermark[zeta] == '1':
            data.at[index,'Cover_Type'] = new_x

            second_hash = hashlib.blake2b(f"{new_x}{k}{abs(x - new_x)}".encode()).hexdigest()
            # print(f"{new_x}{k}{abs(x - new_x)}")
            # print(second_hash)

            record_table[k_hash] = second_hash

        else:
            second_hash = hashlib.blake2b(f"{x}{k}".encode()).hexdigest()

            record_table[k_hash] = second_hash
        # cnt += 1
        # if cnt == 3:
        #     break
        # break

data_file_path = '/home/zhengyihao/dataset/covertype/semanticmark_covertype.data.csv'
data.to_csv(data_file_path, index=False)

400


In [12]:
file_path = '/home/zhengyihao/dataset/covertype/semanticmark_covertype.data.csv'

# 使用列名列表读取数据
data = pd.read_csv(file_path)

# 获取Cover_Type的值域（唯一值）
unique_values = data['Cover_Type'].unique()

# 获取每个值的计数
value_counts = data['Cover_Type'].value_counts()

# 打印结果
print("Cover_Type的值域: ", unique_values)
print("\n每个值的大小:")
print(value_counts)

file_path = '/home/zhengyihao/dataset/covertype/covtype_with_key.data'

# 使用列名列表读取数据
data = pd.read_csv(file_path)

# 获取Cover_Type的值域（唯一值）
unique_values = data['Cover_Type'].unique()

# 获取每个值的计数
value_counts = data['Cover_Type'].value_counts()

# 打印结果
print("原始Cover_Type的值域: ", unique_values)
print("\n每个值的大小:")
print(value_counts)



Cover_Type的值域:  [5 2 1 7 3 6 4]

每个值的大小:
Cover_Type
2    283294
1    211833
3     35768
7     20510
6     17367
5      9493
4      2747
Name: count, dtype: int64
原始Cover_Type的值域:  [5 2 1 7 3 6 4]

每个值的大小:
Cover_Type
2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: count, dtype: int64


## 提取水印

In [55]:
# 数据集文件路径
file_path = '/home/zhengyihao/dataset/covertype/semanticmark_covertype.data.csv'

# 使用列名列表读取数据
data = pd.read_csv(file_path)

count = [[0 for _ in range(2)] for _ in range(L)]
# print(count)
cnt = 0
for index, row in data.iterrows():
    k_hash = hashlib.blake2b(f"{row['primary_key']}{k}".encode()).hexdigest()
    Ks_hash = hashlib.blake2b(f"{Ks}{k_hash}".encode()).hexdigest()


    C = int(Ks_hash,16)
    if C % gamma == 0:
        # zeta = int(Ks_hash,16) % L
        zeta = int(hashlib.blake2b(f"{Ks}{k_hash}{1}".encode()).hexdigest(),16) % L
        x = row['Cover_Type']
        # print(C)
        # print(x)
        # # print(k)
        # print(zeta)
        # print(record_table[k_hash])
        # determine the watermark bit
        if x == 1 and record_table[k_hash] ==  hashlib.blake2b(f"{1}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 2 and record_table[k_hash] ==  hashlib.blake2b(f"{2}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 2 and record_table[k_hash] ==  hashlib.blake2b(f"{2}{k}{2}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 3 and record_table[k_hash] ==  hashlib.blake2b(f"{3}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 3 and record_table[k_hash] ==  hashlib.blake2b(f"{3}{k}{2}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 3 and record_table[k_hash] ==  hashlib.blake2b(f"{3}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 4 and record_table[k_hash] ==  hashlib.blake2b(f"{4}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 4 and record_table[k_hash] ==  hashlib.blake2b(f"{4}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 4 and record_table[k_hash] ==  hashlib.blake2b(f"{4}{k}{0}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 5 and record_table[k_hash] ==  hashlib.blake2b(f"{5}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 5 and record_table[k_hash] ==  hashlib.blake2b(f"{5}{k}{0}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 5 and record_table[k_hash] ==  hashlib.blake2b(f"{5}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 6 and record_table[k_hash] ==  hashlib.blake2b(f"{6}{k}".encode()).hexdigest():
            count[zeta][0] += 1
        elif x == 6 and record_table[k_hash] ==  hashlib.blake2b(f"{6}{k}{1}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 6 and record_table[k_hash] ==  hashlib.blake2b(f"{6}{k}{2}".encode()).hexdigest():
            count[zeta][1] += 1
        elif x == 7 and record_table[k_hash] ==  hashlib.blake2b(f"{7}{k}".encode()).hexdigest():
            count[zeta][0] += 1
            # print("********")
        # break
        # cnt += 1
        # if cnt == 3:
        #     break
W_det = ""
for i in range(L):
    if count[i][0] > count[i][1]:
        W_det += '0'
    else:
        W_det += '1'
# print(count[120][1])
print(count)
print(watermark)
print(W_det)

def mismatch_percentage(W, W_det):
    # 首先检查 W 和 W_det 是否长度相同
    if len(W) != len(W_det):
        print('Error: The lengths of W and W_det are not the same!')
        return
    # 计算不匹配度
    count_mismatch = sum(a != b for a, b in zip(W, W_det))
    percentage = count_mismatch / len(W)
    return percentage

print(f'Mismatch Percentage: {mismatch_percentage(watermark, W_det):.2%}')


[[4, 0], [2, 0], [0, 3], [3, 0], [0, 3], [0, 5], [0, 5], [2, 0], [3, 0], [6, 0], [3, 0], [0, 3], [3, 0], [0, 5], [0, 6], [2, 0], [0, 7], [0, 1], [1, 0], [1, 0], [0, 2], [0, 3], [0, 2], [0, 6], [0, 2], [0, 2], [1, 0], [7, 0], [0, 5], [3, 0], [0, 3], [1, 0], [0, 0], [2, 0], [0, 2], [0, 4], [0, 3], [3, 0], [0, 4], [0, 1], [0, 5], [0, 3], [0, 4], [0, 6], [5, 0], [5, 0], [7, 0], [2, 0], [0, 4], [8, 0], [2, 0], [0, 1], [0, 3], [4, 0], [0, 7], [0, 1], [4, 0], [3, 0], [6, 0], [0, 8], [0, 5], [0, 5], [0, 3], [4, 0], [6, 0], [4, 0], [2, 0], [3, 0], [0, 1], [0, 2], [0, 7], [0, 2], [7, 0], [0, 3], [0, 2], [7, 0], [0, 4], [0, 2], [0, 3], [1, 0], [3, 0], [0, 2], [0, 4], [0, 3], [4, 0], [0, 7], [0, 4], [3, 0], [8, 0], [0, 2], [5, 0], [2, 0], [4, 0], [0, 5], [0, 3], [2, 0], [0, 2], [0, 1], [1, 0], [0, 6], [3, 0], [0, 2], [0, 4], [0, 1], [0, 4], [0, 4], [0, 2], [0, 6], [2, 0], [0, 4], [0, 1], [8, 0], [5, 0], [0, 1], [4, 0], [0, 4], [1, 0], [2, 0], [9, 0], [4, 0], [0, 3], [0, 3], [0, 2], [1, 0], [5, 0],